<a href="https://colab.research.google.com/github/NickolosZH/pet_project/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch, torchvision, pathlib, time, os, glob
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F

from collections import defaultdict
from tqdm.auto import tqdm

from google.colab import drive
from PIL import Image

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
## Гиперпараметры
# Путь к папке
drive.mount('/content/drive')
input_dir = '/content/drive/MyDrive/Colab/Colorectal Cancer/PNG'

# Кол-во потоков для DataLoader
NUM_WORKERS = 2

# Размер изображения
SIZE_H = SIZE_W = 150

# Кол-во классов датасета
NUM_CLASSES = 8

# Кол-во эпох
EPOCH_NUM = 30

# Batch_size
BATCH_SIZE = 256

# Размер тестовой выборки
VAL_SIZE = 0.25

# Среднее и стандартное отклонение каналов изображения для нормализации
image_mean = [0.485, 0.456, 0.406]
image_std = [0.229, 0.224, 0.225]

# Last layer (embeddings) size for CNN models
EMBEDDING_SIZE = 128

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Определим трансформер для предобработки изображений перед созданием датасета
transformer = transforms.Compose([
    transforms.Resize((SIZE_H, SIZE_W)),  # Подгоняем изображения под фиксированный размер
    transforms.ToTensor(),  # Конвертируем в тензоры
    transforms.Normalize(image_mean, image_std)  # Нормализуем по каналам
])

In [ ]:
# Получим номера категорий
classes = os.listdir(input_dir)
int_labels = [int(item[:2]) for item in classes]

In [ ]:
# Получим размер датасета
data_len = sum([len(files) for r, d, files in os.walk(input_dir)])

In [ ]:
all_images = []
for address, dirs, files in os.walk(input_dir):
    for name in files:
        all_images.append(os.path.join(address, name))

# all_images

In [ ]:
# Функция-помощник для отображения изображений
def show_img(cat, image_idx):
    """ Показывает изображение выбранной категории по индексу """

    path = input_dir + '/' + labels[cat] + '/'
    name = os.listdir(path)[image_idx]  # Имя файла
    
    image = Image.open(path + name)

    return Image.show()

In [ ]:
img = Image.open(all_images[1]).convert('RGB')
img.getpalette  
# transformer(img)

<bound method Image.getpalette of <PIL.Image.Image image mode=RGB size=150x150 at 0x7F5467AB0F10>>

In [ ]:
# Создаём класс датасета
class Dataset_CR(Dataset):
  """ Датасет изображений колоректального рака """

  def __init__(self, names = all_images, root_dir=input_dir, labels=int_labels, transform=None):
    """
    Аргументы:
      root_dir (string): Директория со всеми изображениями
      labels (list): Номера категорий
      transform (callable, optional): Опциональные преобразования над 
          изображениями
    """
    self.names = all_images
    self.root_dir = root_dir
    self.labels = labels
    self.transform = transform

  # Получаем длину датасета
  def __len__(self):
    return data_len

  # Получаем тензор изображения
  def __getitem__(self, index):
    ID = self.names[index]
    image = Image.open(ID).convert('RGB')

    # Проверка на трансформации
    if self.transform:
        image = self.transform(image)
    
    x_train = image  # Изображение
    y_train = int(ID.split('/')[-2][:2])  # Номер категории
    
    return x_train, y_train


In [ ]:
# Создаём экземпляр класса Датасет
full_dataset = Dataset_CR(names=all_images, 
                          root_dir=input_dir, 
                          labels=int_labels, 
                          transform=transformer)

In [ ]:
len(full_dataset)

5000

In [ ]:
def subset_ind(dataset, ratio: float):
  """ 
  Функция для получения индексов датасета 
    (для разделения на обучающую и
    тестовую выборки)
  """

  return np.random.choice(len(dataset), size=int(ratio*len(dataset)), replace=False)

In [ ]:
# Получаем тестовой выборки
val_idxs = subset_ind(full_dataset, VAL_SIZE)  

# Для обучающей выборки берём индексы, которые не вошли в тестовые
train_dataset = Subset(full_dataset, [i for i in range(len(full_dataset)) if i not in val_idxs])

# Для тестовой выборки берём тестовые индексы
val_dataset = Subset(full_dataset, val_idxs)  

In [ ]:
# Получаем размер выборок
n_train, n_val = len(train_dataset), len(val_dataset)

In [ ]:
# Создаём DataLoaders

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [ ]:
train_iter = iter(train_loader)

images, labels = next(train_iter)

images.size()

torch.Size([256, 3, 150, 150])

In [ ]:
# plt.figure(figsize=(14,7))
# plt.imshow(images[1, 0], cmap='gray')
# plt.show()

In [ ]:
def images_from_batch_gen(batch_gen):
  """
  Функция выводит несколько изображений из DataLoaders
  """

  data_batch, label_batch = next(iter(batch_gen))
  grid_size = (3, 3)
  f, axarr = plt.subplots(*grid_size)
  f.set_size_inches(15, 10)
  class_names = labels

  for i in range(grid_size[0] * grid_size[1]):

    # Читаем изображения из batch и меняем оси [H, W, C] -> [H, W, C]
    batch_image_ndarray = np.transpose(data_batch[i].numpy(), [1, 2, 0])
    # batch_image_ndarray = data_batch[i].numpy()

    # Инвертируем нормализацию
    src = np.clip(image_std * batch_image_ndarray + image_mean, 0, 1)

    # Отображаем сэмплы с подписями
    sample_title = 'Label = %d (%s)' % (label_batch[i], class_names[label_batch[i]])
    axarr[i // grid_size[0], i % grid_size[0]].imshow(src)
    axarr[i // grid_size[0], i % grid_size[0]].set_title(sample_title)
  pass

In [ ]:
# images_from_batch_gen(train_loader)

# Архитектура модели

In [ ]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__() # Наследуем все свойсвтва у nn.Module
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=9, kernel_size=5, padding=2),
#             nn.BatchNorm2d(9),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )

#         self.layer2 = nn.Sequential(
#             nn.Conv2d(9, 27, kernel_size=5, padding=2),
#             nn.BatchNorm2d(27),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )

#         self.fc = nn.Linear(in_features=27*37*37, out_features=NUM_CLASSES)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.view(out.size(0), -1)
#         out = self.fc(out)

#         return out

# model = CNN().to(device)

In [ ]:
# summary(model, (3, SIZE_H, SIZE_W))

In [ ]:
# criterion = nn.CrossEntropyLoss()  
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)  

In [ ]:
# # Обучение модели
# history = []
# curTime = time.time()
# for epoch in range(EPOCH_NUM):
#     for i, (images, labels) in enumerate(train_loader):   
#         images = images.to(device)
#         labels = labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels-1)
#         loss.backward()
#         history.append(loss)
#         optimizer.step()             

#     print('Эпоха: [%d/%d], Ошибка: %.4f' 
#            % (epoch+1, EPOCH_NUM, loss))    

In [ ]:
# %%timeit -r 1
# # Проверка результатов
# model.eval()  # включаем режим проверки

# model.to(device)

# correct = 0
# total = 0
 
# for images, labels in val_loader:
#     images = images.to(device)
#     labels = labels.to(device)
#     outputs = model(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum()
    
# print('Точность для 5000 картинок: %d %%' % (100 * correct // total))

# Шаблон конструктора

In [ ]:
from IPython.display import clear_output

class Runner():
    """Runner for experiments with supervised model."""
    def __init__(self, model, opt, device, checkpoint_name=None):
        self.model = Даже model
        self.opt = opt
        self.device = device
        self.checkpoint_name = checkpoint_name
        
        self.epoch = 0
        self.output = None
        self.metrics = None
        self._global_step = 0
        self._set_events()
        self._top_val_accuracy = -1
        self.log_dict = {
            "train": [],
            "val": [],
            "test": []
        }
    
    def _set_events(self):
        """
        Additional method to initialize variables, which may store logging and evaluation info.
        The implementation below is extremely simple and only provided to help monitor performance.
        """
        self._phase_name = ''
        self.events = {
            "train": defaultdict(list),
            "val": defaultdict(list),
            "test": defaultdict(list)
        }
    
    def _reset_events(self, event_name):
        self.events[event_name] = defaultdict(list)
    
    def forward(self, img_batch, **kwargs):
        """
        Forward method for your Runner.
        Should not be called directly outside your Runner.
        In simple case, this method should only implement your model forward pass.
        It should also return the model predictions and/or other meta info.
        
        Args:
            batch (mapping[str, Any]): dictionary with data batches from DataLoader.
            **kwargs: additional parameters to pass to the model.
        """
        logits = self.model(img_batch)
        output = {
            "logits": logits,
        }
        return output
    
    def run_criterion(self, batch):
        """
        Applies the criterion to the data batch and the model output, saved in self.output.
        
        Args:
            batch (mapping[str, Any]): dictionary with data batches from DataLoader.
        """
        raise NotImplementedError("To be implemented")
    
    def output_log(self):
        """
        Output log using the statistics collected in self.events[self._phase_name].
        Implement this method for logging purposes.
        """
        raise NotImplementedError("To be implemented")
    
    def _run_batch(self, batch):
        """
        Runs batch of data through the model, performing forward pass.
        This implementation performs data passing to necessary device and is adapted to the default pyTorch DataLoader.
        
        Args:
            batch (mapping[str, Any]): dictionary with data batches from DataLoader.
        """
        # split batch tuple into data batch and label batch
        X_batch, y_batch = batch  # Делим кортеж бача на data и label

        # update the global step in iterations over source data
        self._global_step += len(y_batch)  # обновляем глобальный шаг
        
        # move data to target device
        X_batch = X_batch.to(device)  # Переносим data на device
        
        # run the batch through the model
        self.output = self.forward(X_batch)  # Пропускаем батч через модель и сохраняем в output
    
    def _run_epoch(self, loader, train_phase=True, output_log=False, **kwargs):
        """
        Method that runs one epoch of the training process.
        
        Args:
            loader (DataLoader): data loader to iterate
            train_phase (bool): boolean value to determine if this is the training phase.
                Changes behavior for dropout, batch normalization, etc.
        """
        # Train phase
        # enable or disable dropout / batch_norm training behavior
        self.model.train(train_phase)  # Выставляет режим работы модели в фазу трейн или тест
        
        _phase_description = 'Training' if train_phase else 'Evaluation'
        for batch in tqdm(loader, desc=_phase_description, leave=False):  # Достаём батчи из лоадера
            
            # forward pass through the model using preset device
            self._run_batch(batch)  # Запускаем метод _run_batch и пропускаем каждый батч через модель
            
            # train on batch: compute loss and gradients
            with torch.set_grad_enabled(train_phase):  # Если set_grad включён, то
                loss = self.run_criterion(batch)  # считаем loss для каждого батча
            
            # compute backward pass if training phase
            # reminder: don't forget the optimizer step and zeroing the grads
            if train_phase:  # Если обучающая фаза, то реализуем backprop
                loss.backward()  # Считаем градиент по loss
                self.opt.step()  # Делаем шаг оптимизатора
                self.opt.zero_grad()  # Обнуляем градиенты оптимизатора
        
        self.log_dict[self._phase_name].append(np.mean(self.events[self._phase_name]['loss']))  # Сохраняем метрики в словарь для логирования
        
        if output_log:
            self.output_log(**kwargs)
    
    def train(self, train_loader, val_loader, n_epochs, model=None, opt=None, **kwargs):
        """
        Training process method, that runs for n_epochs over train_loader and performs validation using val_loader.
        
        Args:
            train_loader (DataLoader): training set data loader to iterate over
            val_loader (DataLoader): validation set data loader to iterate over
            n_epochs (int): epoch number to train for
            model (Model): torch nn.Module or nested class, that implements the model. Overwrites self.model.
            opt (Optimizer): torch optimizer to be used for loss minimization. Overwrites self.opt.
            **kwargs: additional parameters to pass to self.validate.

            Метод позволяет обновить train или val loader, кол-во эпох, поменять модель или оптимайзер

        """
        self.opt = (opt or self.opt)  # Оптимизатор
        self.model = (model or self.model)  # Модель
        
        for _epoch in range(n_epochs):  # Проходим по циклу n эпох раз
            start_time = time.time()
            self.epoch += 1
            print(f"epoch {self.epoch:3d}/{n_epochs:3d} started")  # Вывести время обучения
            
            # training part
            self._set_events()
            self._phase_name = 'train'
            self._run_epoch(train_loader, train_phase=True)  # Метод для запуска обучения
            
            print(f"epoch {self.epoch:3d}/{n_epochs:3d} took {time.time() - start_time:.2f}s")
            
            # validation part. Метод проходит по val_loader 
            self._phase_name = 'val'
            self.validate(val_loader, **kwargs)
            self.save_checkpoint()  # и сохраняет чек-пойнты
    
    @torch.no_grad() # we do not need to save gradients during validation
    def validate(self, loader, model=None, phase_name='val', **kwargs):
        """
        Validation process method, that estimates the performance of self.model on validation data in loader.
        
        Args:
            loader (DataLoader): validation set data loader to iterate over
            model (Model): torch nn.Module or nested class, that implements the model. Overwrites self.model.
            opt (Optimizer): torch optimizer to be used for loss minimization. Overwrites self.opt.
            **kwargs: additional parameters to pass to self.validate.
        """
        self._phase_name = phase_name
        self._reset_events(phase_name)
        self._run_epoch(loader, train_phase=False, output_log=True, **kwargs)  # Можно запустить раннер по другому лоадеру
        return self.metrics

# Редактируем шаблон под себя

In [ ]:
from torch.nn.modules.loss import CrossEntropyLoss
from sklearn.metrics import f1_score, accuracy_score

class CNNRunner(Runner):
    def run_criterion(self, batch):
        """
        Applies the criterion to the data batch and the model output, saved in self.output.
        
        Args:
            batch (mapping[str, Any]): dictionary with data batches from DataLoader.
        """
        X_batch, label_batch = batch
        label_batch = label_batch.to(device)
        
        logit_batch = self.output['logits']
        
        # compute loss funciton
        loss = CrossEntropyLoss()(logit_batch, label_batch)  # Сохраняем loss
        
        scores = F.softmax(logit_batch, 1).detach().cpu().numpy()[:, 1].tolist()
        labels = label_batch.detach().cpu().numpy().ravel().tolist()
        
        # log some info. Пишем логи
        self.events[self._phase_name]['loss'].append(loss.detach().cpu().numpy())
        self.events[self._phase_name]['scores'].extend(scores)
        self.events[self._phase_name]['labels'].extend(labels)
        
        return loss
    
    def save_checkpoint(self):  # Сохраняем наилучший чекпойнт
        val_accuracy = self.metrics['accuracy']
        # save checkpoint of the best model to disk
        if val_accuracy > self._top_val_accuracy and self.checkpoint_name is not None:
            self._top_val_accuracy = val_accuracy
            torch.save(model, open(self.checkpoint_name, 'wb'))
    
    def output_log(self, **kwargs):
        """
        Output log using the statistics collected in self.events[self._phase_name].
        Let's have a fancy code for classification metrics calculation.
        """
        scores = np.array(self.events[self._phase_name]['scores'])
        labels = np.array(self.events[self._phase_name]['labels'])
        
        assert len(labels) > 0, print('Label list is empty')
        assert len(scores) > 0, print('Score list is empty')
        assert len(labels) == len(scores), print('Label and score lists are of different size')
        
        visualize = kwargs.get('visualize', False)
        if visualize:
            clear_output()
        
        self.metrics = {
            "loss": np.mean(self.events[self._phase_name]['loss']),
            "accuracy": accuracy_score(labels, np.int32(scores > 0.5)),
            "f1": f1_score(labels, np.int32(scores > 0.5))
        }
        print(f'{self._phase_name}: ', end='')
        print(' | '.join([f'{k}: {v:.4f}' for k, v in self.metrics.items()]))
        
        self.save_checkpoint() 
        
        if visualize:
            # tensorboard for the poor
            fig = plt.figure(figsize=(15,5))
            ax1 = fig.add_subplot(1,2,1)
            ax2 = fig.add_subplot(1,2,2)
            
            ax1.plot(self.log_dict['train'], color='b', label='train')
            ax1.plot(self.log_dict['val'], color='c', label='val')
            ax1.legend()
            ax1.set_title('Train/val loss.')
            
            class_0_scores = np.array(scores)[np.array(labels) == 0]
            class_1_scores = np.array(scores)[np.array(labels) == 1]
            ax2.hist(class_0_scores, bins=50, range=[0,1.01], color='r', alpha=0.7, label='cats')
            ax2.hist(class_1_scores, bins=50, range=[0,1.01], color='g', alpha=0.7, label='dogs')
            ax2.legend()
            ax2.set_title('Validation set score distribution.')
            
            plt.show()

# Создаём модель

In [ ]:
class ConvNet(nn.Module): 
    def __init__(self): 
        super(ConvNet, self).__init__() 
        self.layer1 = nn.Sequential( 
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) 

        self.layer2 = nn.Sequential( 
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) 

        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(64 * 37 * 37, 1000) 
        self.fc2 = nn.Linear(1000, NUM_CLASSES+1)

    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = self.fc1(out) 
        out = self.fc2(out) 
        return out


In [ ]:
model = ConvNet()

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
opt.zero_grad()
ckpt_name = 'model_base.ckpt'
model = model.to(device)

In [ ]:
summary(model, (3, SIZE_H, SIZE_W))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 150, 150]           2,432
              ReLU-2         [-1, 32, 150, 150]               0
         MaxPool2d-3           [-1, 32, 75, 75]               0
            Conv2d-4           [-1, 64, 75, 75]          51,264
              ReLU-5           [-1, 64, 75, 75]               0
         MaxPool2d-6           [-1, 64, 37, 37]               0
           Dropout-7                [-1, 87616]               0
            Linear-8                 [-1, 1000]      87,617,000
            Linear-9                    [-1, 9]           9,009
Total params: 87,679,705
Trainable params: 87,679,705
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 19.20
Params size (MB): 334.47
Estimated Total Size (MB): 353.93
--------------------------------

# Tensorboard

In [ ]:
%load_ext tensorboard
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/run1')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# # get some random training images
# dataiter = iter(train_loader)
# images, labels = next(dataiter)

# # create grid of images
# img_grid = torchvision.utils.make_grid(images)

# # show images
# matplotlib_imshow(img_grid, one_channel=True)

# # write to tensorboard
# writer.add_image('four_colorectal_cancer_images', img_grid)

# writer.add_graph(model, images)

In [ ]:
# helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        # ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
        #     classes[preds[idx]],
        #     probs[idx] * 100.0,
        #     classes[labels[idx]]),
        #             color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# running_loss = 0.0
# for epoch in range(30):  # loop over the dataset multiple times

#     for i, data in enumerate(train_loader, 0):

#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         # if i % 1000 == 999:    # every 1000 mini-batches...

#         # ...log the running loss
#         writer.add_scalar('training loss',
#                         running_loss / 1000,
#                         epoch * len(train_loader) + i)

#         # ...log a Matplotlib Figure showing the model's predictions on a
#         # random mini-batch
#         writer.add_figure('predictions vs. actuals',
#                         plot_classes_preds(model, inputs, labels),
#                         global_step=epoch * len(train_loader) + i)
#         running_loss = 0.0
# print('Finished Training')

In [ ]:
# writer.close()

In [ ]:
# %tensorboard --logdir=runs

# Обучаем модель

In [ ]:
runner = CNNRunner(model, opt, device, ckpt_name)

In [ ]:
# runner.train(train_loader, val_loader, n_epochs=2, visualize=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Задаем параметры нейронной сети
num_epochs = 30
batch_size = BATCH_SIZE
learning_rate = 0.001

# Задаем модель нейронной сети на базе ResNet
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
model.to(device)

# Заменяем последний слой на выход с 9 классами
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 9)
model.to(device)

# Задаем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение нейронной сети
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Передача данных на устройство CUDA
        images = images.to(device)
        labels = labels.to(device)

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Обратный проход и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Оценка модели на тестовых данных
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        # Передача данных на устройство CUDA
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


Accuracy of the network on the 1250 test images: 90.64 %
